# Predict with LoRA

In [29]:
from model_utils_bert import BertRnn, BertRnnDist
from transformers import Trainer, TrainingArguments, BertConfig
from transformers import  AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from dataloader_bert import DataSetLoaderBERT
from utils import compute_metrics
import json
from transformers import TextClassificationPipeline
import torch
from transformers import set_seed
set_seed(10)

Load model and tokenizer

In [30]:
model_name = "/M2/ArgosMHC_models/checkpoints/lora_t33_c3_2/checkpoint-150000/"  # mejor checkpoiunt
name_results = "predictions_esm2_lora_t33_c3" # 
pre_trained = "/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(pre_trained)

#In case you have added tokens, it’s recommended to use the PeftModel class rather than AutoModelForCausalLM. The former takes into account resizing the embedding matrix.

Some weights of the model checkpoint at /M2/ArgosMHC_models/checkpoints/lora_t33_c3_2/checkpoint-150000/ were not used when initializing BertForSequenceClassification: ['base_model.model.bert.encoder.layer.19.attention.output.LayerNorm.bias', 'base_model.model.bert.encoder.layer.26.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.25.attention.self.query.base_layer.weight', 'base_model.model.bert.encoder.layer.14.attention.output.LayerNorm.bias', 'base_model.model.bert.encoder.layer.23.attention.output.dense.weight', 'base_model.model.bert.encoder.layer.12.attention.output.LayerNorm.bias', 'base_model.model.bert.encoder.layer.4.attention.self.query.lora_B.default.weight', 'base_model.model.bert.encoder.layer.16.attention.self.query.lora_A.default.weight', 'base_model.model.bert.encoder.layer.12.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.21.intermediate.dense.weight', 'base_model.model.bert.encoder.layer.17.attention

Load dataset

In [32]:

seq_length = 50 # for MHC-I
test_dataset = DataSetLoaderBERT("/M2/ArgosMHC_models/dataset/hlab/hlab_test_micro.csv", tokenizer_name="/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D", max_length=seq_length)

In [40]:
#for sample in test_dataset:
#    print(sample)
#sample = test_dataset[0]
#print(sample)
#print(sample['input_ids'])
#print( next(iter(test_dataset)) )

print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL", padding='max_length', max_length=seq_length) )
print(test_dataset[0])

print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL", padding='max_length', max_length=seq_length) )
print(test_dataset[1])

#Genera estas diferencia cuando en el tokenizer no definimos que haga un padding 
""""
tokenizer 'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2],
dataloader'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2,  1,  1,  1,  1,  1,  1,  1,  1],

 
tokenizer 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
dataloader'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
"""


{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}
{'input_ids': tensor([ 0, 19, 18,  5, 20, 19, 16,  9, 17, 20,  5, 21, 11, 13,  5, 17, 11,  4,
        19, 12, 12, 19, 10, 13, 19, 11, 22,  7,  5, 10,  7, 19, 10,  6, 19,  4,
        18,  6, 10, 13,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0]), 'labels': tensor(1)}
{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 11, 13, 15, 15, 11, 21, 4, 2, 1, 1, 

'"\ntokenizer \'input_ids\': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2],\ndataloader\'input_ids\': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2,  1,  1,  1,  1,  1,  1,  1,  1],\n\n \ntokenizer \'attention_mask\': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]\ndataloader\'attention_mask\': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]\n'

Pipeline

In [44]:
#tokenizer.padding='max_length'
#tokenizer.max_length=seq_length
print(tokenizer)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL")) # label 1
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL")) # label 1
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYDCEKAFFKM")) # label 0
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYASDDGSWWD")) # label 0

PreTrainedTokenizer(name_or_path='/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/', vocab_size=33, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'})
[[{'label': 'LABEL_0', 'score': 0.3711804747581482}, {'label': 'LABEL_1', 'score': 0.628819465637207}]]
[[{'label': 'LABEL_0', 'score': 0.3667409121990204}, {'label': 'LABEL_1', 'score': 0.6332590579986572}]]
[[{'label': 'LABEL_0', 'score': 0.3783145546913147}, {'label': 'LABEL_1', 'score': 0.6216854453086853}]]
[[{'label': 'LABEL_0', 'score': 0.375162273645401}, {'label': 'LABEL_1', 'score': 0.6248377561569214}]]


# Predict Classic

In [5]:
from model_utils_bert import BertRnn, BertRnnDist
from transformers import Trainer, TrainingArguments, BertConfig
from transformers import  AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from dataloader_bert import DataSetLoaderBERT
from utils import compute_metrics
import json
from transformers import TextClassificationPipeline, pipeline, BartForSequenceClassification
import torch
from transformers import set_seed
set_seed(10)

In [13]:
model_name = "/M2/ArgosMHC_models/checkpoints/classic_t33_c5/checkpoint-153000/"  # mejor checkpoiunt
name_results = "predictions_esm2_t33_c5" # 
pre_trained = "/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/"

model = BertRnn.from_pretrained(model_name, num_labels=2) # it fail for automodel for sequence classification
#model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2) # it fail for automodel for sequence classification
tokenizer = AutoTokenizer.from_pretrained(pre_trained)

In [18]:
seq_length = 50 # for MHC-I
test_dataset = DataSetLoaderBERT("/M2/ArgosMHC_models/dataset/hlab/hlab_test_micro.csv", tokenizer_name="/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D", max_length=seq_length)
print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL", padding='max_length', max_length=seq_length) )
print(test_dataset[0])
print(type(test_dataset[0]))

print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL", padding='max_length', max_length=seq_length) )
print(test_dataset[1])

{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}
{'input_ids': tensor([ 0, 19, 18,  5, 20, 19, 16,  9, 17, 20,  5, 21, 11, 13,  5, 17, 11,  4,
        19, 12, 12, 19, 10, 13, 19, 11, 22,  7,  5, 10,  7, 19, 10,  6, 19,  4,
        18,  6, 10, 13,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0]), 'labels': tensor(1)}
<class 'dict'>
{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 11, 13, 15, 15, 11, 2

In [24]:
# el tokenizer devuelve una lsita, falta convertirlos a tensores
# sample = tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL", padding='max_length', max_length=seq_length)
sample = test_dataset[0]
print( sample)
print( type(sample['input_ids']) )
output = model( sample['input_ids'], sample['attention_mask'] )
prediction = torch.argmax(output)


# genera errores raros, vamos a ver cual es la entrada al forward del modelo, al entrenar y comprar con eso.
# luego de eso degería salir

{'input_ids': tensor([ 0, 19, 18,  5, 20, 19, 16,  9, 17, 20,  5, 21, 11, 13,  5, 17, 11,  4,
        19, 12, 12, 19, 10, 13, 19, 11, 22,  7,  5, 10,  7, 19, 10,  6, 19,  4,
        18,  6, 10, 13,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0]), 'labels': tensor(1)}
<class 'torch.Tensor'>


ValueError: not enough values to unpack (expected 2, got 1)

In [10]:
#tokenizer.padding='max_length'
#tokenizer.max_length=seq_length
print(tokenizer)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
#pipe = pipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
print(pipe(["YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL"])) # label 1
#print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL")) # label 1
#print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYDCEKAFFKM")) # label 0
#print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYASDDGSWWD")) # label 0

/home/vicente/anaconda3/envs/torch11/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
The model 'BertModel' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'EsmForSequenceClassification', 

PreTrainedTokenizer(name_or_path='/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/', vocab_size=33, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'})


KeyError: 'logits'